In [18]:
import urllib
import sqlite3
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# Create a list of words to ignore 
ignorewords=set(['the','of','to','and','a','in','is','it'])

class crawler :
    def __init__(self,dbname):
        self.con=sqlite3.connect(dbname)
        
    def __del__(self):
        self.con.close()
        
    def dbcommit(self):
        self.con.commit()
    
    #help-func to get an identifier  and add an item, if its not
    def getentryid(self,table,field,value,createnew=True):
        cur=self.con.execute(
        "select rowid from %s where %s='%s'" % (table,field,value))
        res=cur.fetchone( )
        if res==None:
            cur=self.con.execute(
            "insert into %s (%s) values ('%s')" % (table,field,value))
            return cur.lastrowid
        else:
            return res[0]
    
    #indexing page
    def addtoindex(self,url,soup):
        if self.isindexed(url): return
        print ('Indexing ', url)
        # Get the individual words
        text=self.gettextonly(soup)
        words=self.separatewords(text)
        # Get the URL id
        urlid=self.getentryid('urllist','url',url)
        # Link each word to this url
        for i in range(len(words)):
            word=words[i]
            if word in ignorewords: continue
            wordid=self.getentryid('wordlist','word',word)
            self.con.execute("insert into wordlocation(urlid,wordid,location) \
                values (%d,%d,%d)" % (urlid,wordid,i))
    
    #retrieving text from a page(without <tag>)
    def gettextonly(self,soup):
        v=soup.string
        if v==None:
            c=soup.contents
            resulttext=''
            for t in c:
                subtext=self.gettextonly(t)
                resulttext+=subtext+'\n'
            return resulttext
        else:
            return v.strip( )
    
    #splitting text into words
    def separatewords(self,text):
        splitter=re.compile('\\W*')
        return [s.lower( ) for s in splitter.split(text) if s!='']
    
    #return true if url is indexed
    def isindexed(self,url):
        u=self.con.execute \
            ("select rowid from urllist where url='%s'" % url).fetchone( )
        if u!=None:
            # Check if it has actually been crawled
            v=self.con.execute(
            'select * from wordlocation where urlid=%d' % u[0]).fetchone( )
            if v!=None: return True
        return False
    
    # Add a link between two pages
    def addlinkref(self,urlFrom,urlTo,linkText):
        pass
    
    # Starting with a list of pages, indexing pages
    def crawl(self,pages,depth=2):
        for i in range(depth):
            newpages=set( )
            for page in pages:
                try:
                    c=urllib.request.urlopen(page)
                except:
                    print("Could not open " , page)
                    continue
                soup=BeautifulSoup(c.read( ))
                self.addtoindex(page,soup)
                links=soup('a')
                for link in links:
                    if ('href' in dict(link.attrs)):
                        url=urljoin(page,link['href'])
                        if url.find("'")!=-1: continue
                        url=url.split('#')[0] # remove location portion
                        if url[0:4]=='http' and not self.isindexed(url):
                            newpages.add(url)
                        linkText=self.gettextonly(link)
                        self.addlinkref(page,url,linkText)
                self.dbcommit( )
            pages=newpages
    
    # creating the database tables
    def createindextables(self):
        self.con.execute('create table urllist(url)')
        self.con.execute('create table wordlist(word)')
        self.con.execute('create table wordlocation(urlid,wordid,location)')
        self.con.execute('create table link(fromid integer,toid integer)')
        self.con.execute('create table linkwords(wordid,linkid)')
        self.con.execute('create index wordidx on wordlist(word)')
        self.con.execute('create index urlidx on urllist(url)')
        self.con.execute('create index wordurlidx on wordlocation(wordid)')
        self.con.execute('create index urltoidx on link(toid)')
        self.con.execute('create index urlfromidx on link(fromid)')
        self.dbcommit( )

        

class searcher:
    def __init__(self,dbname):
        self.con=sqlite3.connect(dbname)
        
    def __del__(self):
        self.con.close( )
    
    def getmatchrows(self,q):
        # Strings to build the query
        fieldlist='w0.urlid'
        tablelist=''
        clauselist=''
        wordids=[]
        # Split the words by spaces
        words=q.split(' ')
        tablenumber=0
        for word in words:
            # Get the word ID
            wordrow=self.con.execute( "select rowid from wordlist where word='%s'" % word).fetchone( )
            if wordrow!=None:
                wordid=wordrow[0]
                wordids.append(wordid)
                if tablenumber>0:
                    tablelist+=','
                    clauselist+=' and '
                    clauselist+='w%d.urlid=w%d.urlid and ' % (tablenumber-1,tablenumber)
                fieldlist+=',w%d.location' % tablenumber
                tablelist+='wordlocation w%d' % tablenumber
                clauselist+='w%d.wordid=%d' % (tablenumber,wordid)
                tablenumber+=1
        # Create the query from the separate parts
        fullquery='select %s from %s where %s' % (fieldlist,tablelist,clauselist)
        cur=self.con.execute(fullquery)
        rows=[row for row in cur]
        return rows,wordids
    
    def getscoredlist(self,rows,wordids):
        totalscores=dict([(row[0],0) for row in rows])
        # This is where you'll later put the scoring functions
        weights=[(1.0,self.frequencyscore(rows)),
                (1.5,self.locationscore(rows)),
                (1.0,self.distancescore(rows))
                ]
        for (weight,scores) in weights:
            for url in totalscores:
                totalscores[url]+=weight*scores[url]
                
        return totalscores
    def geturlname(self,id):
        return self.con.execute(
        "select url from urllist where rowid=%d" % id).fetchone( )[0]
    
    def query(self,q):
        rows,wordids=self.getmatchrows(q)
        scores=self.getscoredlist(rows,wordids)
        rankedscores=sorted([(score,url) for (url,score) in scores.items( )],reverse=1)
        for (score,urlid) in rankedscores[0:10]:
            print('%f\t%s' , (score,self.geturlname(urlid)))
            
    def normalizescores(self,scores,smallIsBetter=0):
        vsmall=0.00001 # Avoid division by zero errors
        if smallIsBetter:
            minscore=min(scores.values( ))
            return dict([(u,float(minscore)/max(vsmall,l)) for (u,l)  in scores.items( )])
        else:
            maxscore=max(scores.values( ))
            if maxscore==0: maxscore=vsmall
            return dict([(u,float(c)/maxscore) for (u,c) in scores.items( )])
    
    def frequencyscore(self,rows):
        counts=dict([(row[0],0) for row in rows])
        for row in rows: counts[row[0]]+=1
        return self.normalizescores(counts)

    def locationscore(self,rows):
        locations=dict([(row[0],1000000) for row in rows])
        for row in rows:
            loc=sum(row[1:])
            if loc<locations[row[0]]: locations[row[0]]=loc
        return self.normalizescores(locations,smallIsBetter=1)
    
    def distancescore(self,rows):
        # If there's only one word, everyone wins!
        if len(rows[0])<=2: return dict([(row[0],1.0) for row in rows])
        # Initialize the dictionary with large values
        mindistance=dict([(row[0],1000000) for row in rows])
        for row in rows:
            dist=sum([abs(row[i]-row[i-1]) for i in range(2,len(row))])
            if dist<mindistance[row[0]]: mindistance[row[0]]=dist
        return self.normalizescores(mindistance,smallIsBetter=1)

In [63]:
eq = search.con.execute("select * from urllist ").fetchall()

In [66]:
weq

[('https://www.coursera.org',),
 ('https://www.coursera.org/learn/excel-para-negocios',),
 ('https://www.coursera.org/?authMode=signup',),
 ('https://itunes.apple.com/app/apple-store/id736535961?pt=2334150&ct=Coursera%20Web%20Promo%20Banner&mt=8',),
 ('https://www.coursera.org/directory',),
 ('https://www.coursera.org/featured/top_rated_specializations',),
 ('https://about.coursera.org/leadership',),
 ('https://building.coursera.org/developer-program/',),
 ('https://www.coursera.org/learn/matlab',),
 ('https://www.coursera.org/specializations/tesol-certificate-2',),
 ('https://www.coursera.org/specializations/data-science-python',),
 ('https://www.coursera.org/specializations/mathematics-machine-learning',),
 ('https://about.coursera.org/affiliates',),
 ('https://www.coursera.org/about/partners',),
 ('https://www.coursera.org/learn/deep-neural-network',),
 ('https://www.coursera.org/partners',),
 ('https://www.coursera.org/courses',),
 ('https://www.coursera.org/learn/programming-langu

In [27]:
crawler = crawler('searchindex.db')


In [19]:
search = searcher('searchindex1.db')

In [20]:
search.query('functional programming')

%f	%s (3.5, 'https://www.coursera.org/learn/programming-languages')
%f	%s (3.3770970003505436, 'https://www.coursera.org/learn/programming-languages-part-b')
%f	%s (2.331285331759194, 'https://www.coursera.org/featured/top_specializations_locale_en_os_web')
%f	%s (0.5450012735064327, 'https://learningnetwork.cisco.com/community/certifications/ccie_wireless/written-exam-v3/exam-topics')
%f	%s (0.537399254573755, 'https://learningnetwork.cisco.com/community/learning_center/ccie-ccde-evolving-technologies')
%f	%s (0.49899425964877253, 'https://learningnetwork.cisco.com/community/certifications/ccie_data_center/lab_exam/exam-topics')
%f	%s (0.32743668990552055, 'https://www.coursera.org/specializations/strategic-leadership')
